In [1]:
#matplotlib inline
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
import pandas as pd
import time
import mpmath
import os
import VMF

import importlib
importlib.reload(VMF)
#import FVMF
#importlib.reload(FVMF)

prefix = "_phoneme_bg_"
# define the summary writer
writer = SummaryWriter()
sns.set()
sns.set_style("dark")
sns.set_palette("muted")
sns.set_color_codes("muted")


# select the device
DEVICE = torch.device("cuda:1")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
cuda = torch.cuda.set_device(1)

if (torch.cuda.is_available()):
    print("GPUs are used!")
else:
    print("CPUs are used!")

# define the parameters
BATCH_SIZE = 100
TEST_BATCH_SIZE = 100
batch_size = 100
COND_OPT = False
CLASSES = 5
# TRAIN_EPOCHS = 250
SAMPLES = 1
TEST_SAMPLES = 10
TEMPER = 0.001
TEMPER_PRIOR = 0.001
epochs = 250
pepochs = 50

#prepare the data
data = pd.read_csv('http://www.uio.no/studier/emner/matnat/math/STK2100/data/phoneme.data')
data = data.drop(columns=["row.names"])
data = pd.concat([data,data.g.astype("category").cat.codes.astype(int)],sort=False, axis=1) #get_dummies(data['g'], prefix='phoneme')],sort=False, axis=1)
data = data.drop(columns=["g","speaker"])
data = data.values


np.random.seed(40590)

tr_ids = np.random.choice(4509, 3500, replace = False)
te_ids = np.setdiff1d(np.arange(4509),tr_ids)[0:1000]

dtrain = data[tr_ids,:]

data_mean = dtrain.mean(axis=0)[0:256]
data_std = dtrain.std(axis=0)[0:256]

data[:,0:256] = (data[:,0:256]  - data_mean)/data_std




dtrain = data[tr_ids,:]
dtest = data[te_ids,:]


TRAIN_SIZE = len(tr_ids)
TEST_SIZE = len(te_ids)
NUM_BATCHES = TRAIN_SIZE/BATCH_SIZE
NUM_TEST_BATCHES = len(te_ids)/BATCH_SIZE

# set prior parameters
PI = 1
SIGMA_1 = torch.cuda.FloatTensor([math.exp(-0)])
SIGMA_2 = torch.cuda.FloatTensor([math.exp(-6)])


#The net does not like to get larger at a given layer??
l1shape=(256, 5)
l2shape=(5, 5)
l3shape=(5, 5)
l4shape=(5, 5)
layershapes = [l1shape, l2shape, l3shape, l4shape]

epochs = 40
trtimes  = np.zeros(epochs)
# make inference on 10 networks
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net = VMF.BayesianNetwork(l1=l1shape, l2=l2shape, l3=l3shape,l4=l4shape,BN='notbatchnorm').to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr=0.0007)
    for epoch in range(epochs):

        trtimes[epoch] = VMF.train(net, optimizer, epoch, i)
        print(net.l1.weight_mu.mean())

    res = VMF.test_ensemble(net)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

GPUs are used!
Classes loaded
GPUs are used!
Classes loaded
GPUs are used!
0


/opt/uio/modules/rhel8/easybuild/software/Miniconda3/lmsunde/envs/BNN2/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


1
tensor(314.3969, device='cuda:1', grad_fn=<AddBackward0>)
tensor(161.8000, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0103, device='cuda:1', grad_fn=<MeanBackward0>)
2
tensor(310.1597, device='cuda:1', grad_fn=<AddBackward0>)
tensor(157.1232, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0199, device='cuda:1', grad_fn=<MeanBackward0>)
3
tensor(294.5299, device='cuda:1', grad_fn=<AddBackward0>)
tensor(143.0686, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0316, device='cuda:1', grad_fn=<MeanBackward0>)
4
tensor(275.1546, device='cuda:1', grad_fn=<AddBackward0>)
tensor(125.0134, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0364, device='cuda:1', grad_fn=<MeanBackward0>)
5
tensor(261.3079, device='cuda:1', grad_fn=<AddBackward0>)
tensor(112.2422, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0415, device='cuda:1', grad_fn=<MeanBackward0>)
6
tensor(251.6520, device='cuda:1', grad_fn=<AddBackward0>)
tensor(102.4753, device='cuda:1', grad_fn=<Nl

In [2]:
print('net.l1.weight_mu', '\n', 'Max:', torch.max(net.l1.weight_mu), 'Min:',torch.min(net.l1.weight_mu))
print('net.l1.bias_mu', '\n', 'Max:', torch.max(net.l1.bias_mu), 'Min:',torch.min(net.l1.weight_mu))

w_mu1_nodewise = net.l1.weight_mu
w_mu1 = w_mu1_nodewise.reshape(l1shape[0]*l1shape[1]).to(DEVICE)
#net.l1.weight_rho
b_mu1 = net.l1.bias_mu.to(DEVICE) #400
#net.l1.bias_rho

w_mu2_nodewise = net.l2.weight_mu
w_mu2 = w_mu2_nodewise.reshape(l2shape[0]*l2shape[1]).to(DEVICE)
#net.l2.weight_rho
b_mu2 = net.l2.bias_mu.to(DEVICE) #600
#net.l2.bias_rho

w_mu3_nodewise = net.l3.weight_mu
w_mu3 = w_mu3_nodewise.reshape(l3shape[0]*l3shape[1]).to(DEVICE)
#net.l3.weight_rho
b_mu3 = net.l3.bias_mu.to(DEVICE) #5
#net.l3.bias_rho

w_mu4_nodewise = net.l4.weight_mu
w_mu4 = w_mu4_nodewise.reshape(l4shape[0]*l4shape[1]).to(DEVICE)
#net.l3.weight_rho
b_mu4 = net.l4.bias_mu.to(DEVICE) #5
#net.l3.bias_rho
r"""
w_mu5 = net.l5.weight_mu
w_mu5 = w_mu5.reshape(l3shape[0]*l3shape[1]).to(DEVICE)
#net.l3.weight_rho
b_mu5 = net.l5.bias_mu.to(DEVICE) #5
#net.l3.bias_rho
"""

net.l1.weight_mu 
 Max: tensor(0.7738, device='cuda:1', grad_fn=<MaxBackward1>) Min: tensor(-0.6065, device='cuda:1', grad_fn=<MinBackward1>)
net.l1.bias_mu 
 Max: tensor(0.3240, device='cuda:1', grad_fn=<MaxBackward1>) Min: tensor(-0.6065, device='cuda:1', grad_fn=<MinBackward1>)


'\nw_mu5 = net.l5.weight_mu\nw_mu5 = w_mu5.reshape(l3shape[0]*l3shape[1]).to(DEVICE)\n#net.l3.weight_rho\nb_mu5 = net.l5.bias_mu.to(DEVICE) #5\n#net.l3.bias_rho\n'

In [3]:
#import importlib
#import os
import FVMF
importlib.reload(FVMF)

#import VMF

#import importlib
#importlib.reload(VMF)

prefix = "_phoneme_bg_"
# define the summary writer
writer = SummaryWriter()
sns.set()
sns.set_style("dark")
sns.set_palette("muted")
sns.set_color_codes("muted")


# select the device
DEVICE = torch.device("cuda:1")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
cuda = torch.cuda.set_device(1)

# define the parameters
BATCH_SIZE = 100
TEST_BATCH_SIZE = 100
batch_size = 100
COND_OPT = False
CLASSES = 5
# TRAIN_EPOCHS = 250
SAMPLES = 1
TEST_SAMPLES = 10
TEMPER = 0.001
TEMPER_PRIOR = 0.001
pepochs = 50
epochs = 250

#prepare the data
data = pd.read_csv('http://www.uio.no/studier/emner/matnat/math/STK2100/data/phoneme.data')
data = data.drop(columns=["row.names"])
data = pd.concat([data,data.g.astype("category").cat.codes.astype(int)],sort=False, axis=1) #get_dummies(data['g'], prefix='phoneme')],sort=False, axis=1)
data = data.drop(columns=["g","speaker"])
data = data.values


np.random.seed(40590)

tr_ids = np.random.choice(4509, 3500, replace = False)
te_ids = np.setdiff1d(np.arange(4509),tr_ids)[0:1000]

dtrain = data[tr_ids,:]

data_mean = dtrain.mean(axis=0)[0:256]
data_std = dtrain.std(axis=0)[0:256]

data[:,0:256] = (data[:,0:256]  - data_mean)/data_std




dtrain = data[tr_ids,:]
dtest = data[te_ids,:]


TRAIN_SIZE = len(tr_ids)
TEST_SIZE = len(te_ids)
NUM_BATCHES = TRAIN_SIZE/BATCH_SIZE
NUM_TEST_BATCHES = len(te_ids)/BATCH_SIZE

# set prior parameters
PI = 1
SIGMA_1 = torch.cuda.FloatTensor([math.exp(-0)])
SIGMA_2 = torch.cuda.FloatTensor([math.exp(-6)])


epochs = 10
trtimes  = np.zeros(epochs)
w_mu = [w_mu1, w_mu2, w_mu3, w_mu4]
b_mu = [b_mu1, b_mu2, b_mu3, b_mu4]

w_mu_nodewise = [w_mu1_nodewise,w_mu2_nodewise,w_mu3_nodewise,w_mu4_nodewise]
#b_mu_nodewise = [b_mu1_nodewise,b_mu2_nodewise,b_mu3_nodewise,b_mu4_nodewise]
# make inference on 10 networks

for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net2 = FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                VD='vmf',
                                b_kappa=torch.Tensor(1).uniform_(4,4.1),
                                w_kappa=torch.Tensor(1).uniform_(8,8.1),
                                Temper = 1)
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net2.parameters(), lr=0.14)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net2, optimizer, epoch, i)
        #print(net2.l1.weight_mu.mean())

    res = FVMF.test_ensemble(net2)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")


FVMF RELOADED
GPUs are used!
Classes loaded
FVMF RELOADED
GPUs are used!
Classes loaded
0


/uio/hume/student-u44/lmsunde/projects/BNN/AliaksandrFolder/FVMF.py:282: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v0 = torch.cat([v0, torch.tensor(w0[det >= 0]).to(DEVICE)])
/opt/uio/modules/rhel8/easybuild/software/Miniconda3/lmsunde/envs/BNN2/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


1
tensor(146.3880, device='cuda:1', grad_fn=<AddBackward0>)
tensor(46.8615, device='cuda:1', grad_fn=<NllLossBackward0>)
2
tensor(148.6208, device='cuda:1', grad_fn=<AddBackward0>)
tensor(48.4478, device='cuda:1', grad_fn=<NllLossBackward0>)
3
tensor(148.3137, device='cuda:1', grad_fn=<AddBackward0>)
tensor(48.0933, device='cuda:1', grad_fn=<NllLossBackward0>)
4
tensor(147.4099, device='cuda:1', grad_fn=<AddBackward0>)
tensor(46.0398, device='cuda:1', grad_fn=<NllLossBackward0>)
5
tensor(148.8202, device='cuda:1', grad_fn=<AddBackward0>)
tensor(48.8259, device='cuda:1', grad_fn=<NllLossBackward0>)
6
tensor(149.8151, device='cuda:1', grad_fn=<AddBackward0>)
tensor(47.6771, device='cuda:1', grad_fn=<NllLossBackward0>)
7
tensor(147.6231, device='cuda:1', grad_fn=<AddBackward0>)
tensor(46.9457, device='cuda:1', grad_fn=<NllLossBackward0>)
8
tensor(150.9608, device='cuda:1', grad_fn=<AddBackward0>)
tensor(49.8111, device='cuda:1', grad_fn=<NllLossBackward0>)
9
tensor(148.4559, device='cuda:

In [4]:
optimizer = optim.Adam(net2.parameters(), lr=0.07)


for epoch in range(epochs):
    trtimes[epoch] = FVMF.train(net2, optimizer, epoch, i)
    #print(net2.l1.weight_mu.mean())
res = FVMF.test_ensemble(net2)
#np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

1
tensor(46.9905, device='cuda:1', grad_fn=<AddBackward0>)
tensor(46.9905, device='cuda:1', grad_fn=<NllLossBackward0>)
2
tensor(49.6683, device='cuda:1', grad_fn=<AddBackward0>)
tensor(49.6683, device='cuda:1', grad_fn=<NllLossBackward0>)
3
tensor(47.9324, device='cuda:1', grad_fn=<AddBackward0>)
tensor(47.9324, device='cuda:1', grad_fn=<NllLossBackward0>)
4
tensor(47.1820, device='cuda:1', grad_fn=<AddBackward0>)
tensor(47.1820, device='cuda:1', grad_fn=<NllLossBackward0>)
5
tensor(46.3849, device='cuda:1', grad_fn=<AddBackward0>)
tensor(46.3849, device='cuda:1', grad_fn=<NllLossBackward0>)
6
tensor(48.2622, device='cuda:1', grad_fn=<AddBackward0>)
tensor(48.2622, device='cuda:1', grad_fn=<NllLossBackward0>)
7
tensor(45.8986, device='cuda:1', grad_fn=<AddBackward0>)
tensor(45.8986, device='cuda:1', grad_fn=<NllLossBackward0>)
8
tensor(46.9790, device='cuda:1', grad_fn=<AddBackward0>)
tensor(46.9790, device='cuda:1', grad_fn=<NllLossBackward0>)
9
tensor(51.0250, device='cuda:1', grad_

In [5]:
optimizer = optim.Adam(net2.parameters(), lr=0.007)

for epoch in range(epochs):
    trtimes[epoch] = FVMF.train(net2, optimizer, epoch, i)
    #print(net2.l1.weight_mu.mean())
res = FVMF.test_ensemble(net2)
#np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

1
tensor(47.4001, device='cuda:1', grad_fn=<AddBackward0>)
tensor(47.4001, device='cuda:1', grad_fn=<NllLossBackward0>)
2
tensor(47.2060, device='cuda:1', grad_fn=<AddBackward0>)
tensor(47.2060, device='cuda:1', grad_fn=<NllLossBackward0>)
3
tensor(46.4351, device='cuda:1', grad_fn=<AddBackward0>)
tensor(46.4351, device='cuda:1', grad_fn=<NllLossBackward0>)
4
tensor(46.3657, device='cuda:1', grad_fn=<AddBackward0>)
tensor(46.3657, device='cuda:1', grad_fn=<NllLossBackward0>)
5
tensor(46.4936, device='cuda:1', grad_fn=<AddBackward0>)
tensor(46.4936, device='cuda:1', grad_fn=<NllLossBackward0>)
6
tensor(48.7319, device='cuda:1', grad_fn=<AddBackward0>)
tensor(48.7319, device='cuda:1', grad_fn=<NllLossBackward0>)
7
tensor(47.8252, device='cuda:1', grad_fn=<AddBackward0>)
tensor(47.8252, device='cuda:1', grad_fn=<NllLossBackward0>)
8
tensor(47.2033, device='cuda:1', grad_fn=<AddBackward0>)
tensor(47.2033, device='cuda:1', grad_fn=<NllLossBackward0>)
9
tensor(45.7435, device='cuda:1', grad_

In [ ]:
r"""
What if the problem is that since the mu's in the Gaussian is (out,in), and the vMF is out*in,
this could mean that we have "strethed" a single vMF pdf over all the parameters, while in the Gaussian we have made one for each output?

I don't know, also intuitively this should be the case, since the whole point of the vMF is the norm 1, which obviously
will require that it for each forward-pass is only one massive pdf for all inputs and outputs.

Ok, this can get rough, I will try my best to reshape the w_mu's and b_mu's the best I can, perhaps this will work just fine without too much tuning...
Hopefully.

This doe not seem to ave worked. I don't konw exactly what the problem is. Per haps it is a good idea to consult the new loss function
suggested in the paper that made the code we based our vMF on?

It is very strange that the loss is not at all affected by completely ridiculous learning rates...

Remember that vMF makes the norm of the weights and biases 1, not the forward pass of the x's. Hence the advantage is that the gradient
will not explode, since the backward pass of it will also be approx. 1. In batchnorm, maybe the gradient can explode? Since the weights 
can be whatever?


IMPORTANT: The gaussian neuralnet will also collapse to 257 if I apply more than 3 layers. This must be somehow related to the similar
behavior in the vMF when the size of each layer exceeds 3. Per haps there is an error in the loss afterall?
However, in the Gaussian case, increasing the learning rate by a factor of 10 solved the issue. This makes me suspect it is the mathematical
properties of the loss function, rather than incorrect implementation.
"""

#When l4 is (3,5):
r"""
File ~/projects/BNN/AliaksandrFolder/FVMF.py:289, in vMF.sample(self, N, rsf)
    287 e1mu = torch.zeros(d, 1).to(DEVICE)
    288 e1mu[0, 0] = 1.0
--> 289 e1mu = e1mu - self.mu if len(self.mu.shape) == 2 else e1mu - self.mu.unsqueeze(1) #e1mu.shape = (1,self.x_dim). mu_unnorm.shape = (mu_unnorm)
    290 e1mu = e1mu / norm(e1mu, dim=0).to(DEVICE)
    291 samples = samples - 2 * (samples @ e1mu) @ e1mu.t()

RuntimeError: The size of tensor a (15) must match the size of tensor b (9) at non-singleton dimension 0
"""

#When l4 is (5,5):
r"""
File ~/projects/BNN/AliaksandrFolder/FVMF.py:289, in vMF.sample(self, N, rsf)
    287 e1mu = torch.zeros(d, 1).to(DEVICE)
    288 e1mu[0, 0] = 1.0
--> 289 e1mu = e1mu - self.mu if len(self.mu.shape) == 2 else e1mu - self.mu.unsqueeze(1) #e1mu.shape = (1,self.x_dim). mu_unnorm.shape = (mu_unnorm)
    290 e1mu = e1mu / norm(e1mu, dim=0).to(DEVICE)
    291 samples = samples - 2 * (samples @ e1mu) @ e1mu.t()

RuntimeError: The size of tensor a (25) must match the size of tensor b (15) at non-singleton dimension 0
"""
#These errors above were caused by my initialization being wrong. I copy paster mu_3 for layer4, and forgot to change to mu_4. So now 
#I always get the error below.


#in all cases now: 

r"""
It seems that the whole thing does not progress at all. We just get the warning and then no further output.

self.l4(x, sample)

--> self.bias.sample()

It always get's stuck there!!

Specifically, it get's stuck in the while loop:

while len(v0) < N:
            eps = beta.sample([1, rsf * (N - len(v0))]).squeeze().to(DEVICE)
            uns = uniform.sample([1, rsf * (N - len(v0))]).squeeze().to(DEVICE)
            w0 = (1 - (1 + bb) * eps) / (1 - (1 - bb) * eps)
            t0 = (2 * aa * bb) / (1 - (1 - bb) * eps)
            det = (d - 1) * t0.log() - t0 + dd - uns.log()
            v0 = torch.cat([v0, torch.tensor(w0[det >= 0]).to(DEVICE)])
            if len(v0) > N:
                v0 = v0[:N]
                break
"""

r"""
From further investigations it is clear that the error lies in w0[det >= 0] consistently being an empty Tensor.

Even further, bb is 0 here which it usually is not. That must definitely indicate something is wrong.

Adjusting the initialization of kappa to be 9 or less on both weights and biases makes the code run, 
but posterior collapse is back. Increasing kappa seems to increase the compute aswell... however, getting the kappa inits
closer to 10 seems to also help avoid the posterior collapse. And the lower bound increased also helps, looks like 3 is optimal.
"""


r"""

It seems that Torch likes that each of the bias_mu's and weight_mu's from each layer are separately registered with name as an nn.parameter.


"""

r"""

Currently, for 4 layers with 3 hiddenwidth, it seems around 10 epochs with .14 learning rate is optimal for testperformance.
I suspect this is because we do not have the modelcapacity to go beyond the overfiting case just yet, and must settle for the classical
best-fit.
"""
